# CoT Explainability for RAG

Dieses Notebook zeigt, wie der Explainability-Prompt mit Retrieval-Trace genutzt wird.

basierend auf: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=11053836&tag=1


In [1]:
import sys
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [2]:
from src.modules.cot_explainable import CoTExplainable, ExplainableAnswer, _format_documents
from src.modules.rag_engine import RAGEngine
from src.modules.llm_client import LLMClient
from src.modules.data_loader import DataLoader

/Users/MeinNotebook/xai-rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Real data example

In [3]:
_dataLoader = DataLoader()
documents = _dataLoader.setup()

✓ Dataset found at /Users/MeinNotebook/xai-rag/data/raw/hotpot_test_fullwiki_v1.json
Loading data into memory...
✓ Loaded 7405 questions.
Converting HotPotQA contexts to documents...


Processing Articles: 100%|██████████| 7405/7405 [00:00<00:00, 33113.30it/s]

✓ Created 73774 context chunks.


In [4]:
ragEngine = RAGEngine()
ragEngine.setup(documents=documents)

Creating new vector store with 73774 documents...
Split into 73826 chunks.


KeyboardInterrupt: 

In [5]:
#print(response['parsed'].answer)
#print(response['parsed'].evidence)
#print(response['parsed'].used_sources)
#print(response['parsed'].reasoning)

In [6]:
from src.modules.multihop_rag_engine import MultiHopRAGEngine

In [7]:
client = LLMClient(provider="ollama", model_name="gemma3:1b")
mh_rag = MultiHopRAGEngine(rag_engine=ragEngine, llm_client=client, num_hops=2)

Connecting to local Ollama (gemma3:1b)...


In [8]:
question = "Which experimental pop band has James Huggins III as a member and belongs to the Elephant 6 collective?"

trace, all_documents = mh_rag.run_and_trace(question)

final_answer = trace['final_answer']

--- Starting Multi-Hop Search for: 'Which experimental pop band has James Huggins III as a member and belongs to the Elephant 6 collective?' ---

[ Hop 1 ]
Executing search with query: 'Which experimental pop band has James Huggins III as a member and belongs to the Elephant 6 collective?'
Generating next query...

[ Hop 2 ]
Executing search with query: 'What is the name of the recording project of James Huggins III?'

Generating final answer...

--- Multi-Hop Search Complete. Final Answer: James Huggins III is the recording project of James Husband, who is the multi-instrumentalist of Of Montreal. ---
--- Multi-Hop Context: 

 ('<doc id="chunk-1-1" from_hop="1" search_query="Which experimental pop band has James Huggins III as a member and belongs to the Elephant 6 collective?">\nJames Husband is the recording project of James Huggins III, Of Montreal\'s multi-instrumentalist.\n</doc>', ['no-source', 'chunk-1-1'])

 ('<doc id="chunk-2-1" from_hop="2" search_query="What is the name of 

In [9]:
explain_client = LLMClient(provider="ollama", model_name="gemma3:1b", structured_output=ExplainableAnswer)

explainer = CoTExplainable(llm_client=explain_client)
response = explainer.explain(query=question, answer=final_answer, documents=all_documents)

Connecting to local Ollama (gemma3:1b)...


In [10]:
print(response.evidence)
print(response.reasoning)

[EvidenceItem(span='James Huggins III is the recording project of James Husband', support=['chunk-1-1'], rationale='chunk-1-1 states James Husband is the recording project of James Huggins III.'), EvidenceItem(span='who is the multi-instrumentalist of Of Montreal', support=['chunk-1-1'], rationale='chunk-1-1 states James Husband is the multi-instrumentalist of Of Montreal.')]
[ReasoningStep(step=1, statement='The question asks for the identity of a specific experimental pop band.', support=['chunk-1-1'], support_type='direct', quote='James Huggins III is the recording project of James Husband, who is the multi-instrumentalist of Of Montreal.')]


In [14]:
print(_format_documents(all_documents, from_query=question))

('<doc id="chunk-1-1" from_hop="1" search_query="Which experimental pop band has James Huggins III as a member and belongs to the Elephant 6 collective?">\nJames Husband is the recording project of James Huggins III, Of Montreal\'s multi-instrumentalist.\n</doc>\n<doc id="chunk-1-2" from_hop="1" search_query="Which experimental pop band has James Huggins III as a member and belongs to the Elephant 6 collective?">\nJames Husband is the recording project of James Huggins III, Of Montreal\'s multi-instrumentalist.\n</doc>', ['no-source', 'chunk-1-1', 'chunk-1-2'])
